In [2]:
import pandas, seaborn, scipy, numpy, matplotlib, collections, itertools, math, functools, sys, sklearn

/Users/tim/miniconda3/envs/py3k/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [3]:
ega_filenames = pandas.read_table("../data/ega_contents.aspera.tsv", header=None)[0]

def lookup_filename(substring):
    results = ega_filenames[ega_filenames.str.contains(substring)]
    if len(results) == 0:
        print("NOT_FOUND:%s" % substring)
        return "NOT_FOUND:%s" % substring
    if len(results) > 1:
        print("MULTIPLE_FOUND:%s" % substring)
        return "MULTIPLE_FOUND:%s" % substring
    return list(results)[0]


In [5]:
ega_filenames

0      EGAZ00001018524_2014_ICGC_IcgcOvarian_AOCS137_...
1      EGAZ00001018525_2014_ICGC_IcgcOvarian_AOCS091_...
2      EGAZ00001018526_2014_ICGC_IcgcOvarian_AOCS057_...
3      EGAZ00001018527_2014_ICGC_130906_D81P8DQ1_0153...
4      EGAZ00001018528_2014_ICGC_130906_D81P8DQ1_0153...
5      EGAZ00001018529_2014_ICGC_140227_EXTERN_0128_A...
6      EGAZ00001018530_2014_ICGC_131206_EXTERN_0045_A...
7      EGAZ00001018531_2014_ICGC_140227_EXTERN_0129_B...
8      EGAZ00001018532_2014_ICGC_IcgcOvarian_AOCS160_...
9      EGAZ00001018533_2014_ICGC_IcgcOvarian_AOCS122_...
10     EGAZ00001018534_2014_ICGC_130906_D81P8DQ1_0157...
11     EGAZ00001018535_2014_ICGC_IcgcOvarian_AOCS077_...
12     EGAZ00001018536_2014_ICGC_140227_EXTERN_0128_A...
13     EGAZ00001018537_2014_ICGC_IcgcOvarian_AOCS064_...
14     EGAZ00001018538_2014_ICGC_IcgcOvarian_AOCS117_...
15     EGAZ00001018539_2014_ICGC_IcgcOvarian_AOCS093_...
16     EGAZ00001018540_2014_ICGC_IcgcOvarian_AOCS088_...
17     EGAZ00001018541_2014_ICG

In [ ]:
samples = pandas.read_table("../data/sample.tsv")
specimens = pandas.read_table("../data/specimen.tsv")
ids = pandas.read_csv("../data/ICGC_IDs_19Oct2015_External_modified.csv")

sources = pandas.merge(ids, samples, left_on='DNA_biospecimen', right_on="submitted_sample_id", how='inner')
sources = pandas.merge(sources, specimens, on="icgc_specimen_id", how='inner').copy()


assert all(sources.submitted_donor_id_x == sources.submitted_donor_id_y)
assert all(sources.icgc_donor_id_x == sources.icgc_donor_id_y)

sources["cohort"] = "AOCS"
sources["donor"] = sources.submitted_donor_id_x
sources["tissue_type"] = sources.SpecimenType.map({"Tumour": "solid", "Ascites": "ascites"})
sources["timepoint"] = sources.CollectionPoint.map(
    {"Primary": "primary", "Recurrence": "recurrence", "Autopsy": "recurrence"})

sources["source_id"] = sources.DNA_biospecimen
sources.index = sources.source_id

sources["treated"] = sources.specimen_donor_treatment_type == "other therapy"
sources["metastasis"] = sources.specimen_type_description.str.contains("metastasis")
sources["interval_days"] = sources.specimen_interval

print('looking for dna')
sources["bam_path_tumor_dna"] = [lookup_filename(row.DNA_sample_string) for (i,row) in sources.iterrows()]

print("looking for rna")
sources["bam_path_tumor_rna"] = [lookup_filename(row["RNA data file"]) for (i,row) in sources.iterrows()]

sources.loc["AOCS-170-1-8", "treated"] = False  # weird incorrect data point based on email from Elizabeth Christie


assert sources["source_id"].nunique() == 114
assert len(sources["source_id"]) == 114

simple_sources = sources[
    "source_id,donor,cohort,treated,timepoint,metastasis,tissue_type,interval_days,bam_path_tumor_dna,bam_path_tumor_rna".split(",")
]

sources.to_csv("../data/sources.full.csv", index=False)
simple_sources.to_csv("../data/sources.simple.csv", index=False)


In [ ]:
simple_sources.tail()

In [ ]:
original_sources = pandas.read_csv("../data/sources.aocs_only.csv")
original_sources
original_sources.treated.value_counts()

In [ ]:
original_sources.donor.nunique()
simple_sources.donor.nunique()

In [ ]:
original_sources.shape, simple_sources.shape

In [ ]:
simple_sources.treated.value_counts(), original_sources.treated.value_counts()

In [ ]:
simple_sources.metastasis.value_counts(), original_sources.metastasis.value_counts()